# Cross-Referencing Guide

Merlin now gives me MRN, PMRN, and Encounter. The MRN is correct in Epic but the PMRN is not.
 
Vesela gave me a file with PMRN and Encounter. This Encounter matches Merlin but the PMRN does not match Merlin, instead the PMRN matches Epic.
 
My colleague Ayumi gave me a file with MRN and Encounter. The MRN is correct in Epic but the Encounter does not match the other Encounters.

To achieve my end goal of matching Merlin's Encounters with Ayumi's Encounters, I need to do the following:
 
1) Start with Ayumi's Encounter
2) Find Ayumi's corresponding MRN
3) Match this MRN to Merlin
4) Find Merlin's corresponding Encounter
 
And then if I want to go from Ayumi's Encounter to a patient's true PMRN, I need to add the following steps:

5) Start with Merlin's Encounter
6) Match this Encounter in Vesela's raw data file
7) Find the corresponding PMRN in the raw data file

In [ ]:
# prompt: Import libraries

import pandas as pd
import numpy as np
import re
import datetime
import matplotlib.pyplot as plt


In [ ]:
my_computer_fpath = "C:\\Users\\User\\OneDrive - Mass General Brigham\\Epidural project\\Data\\"
merlin_df = merlin_df = pd.read_csv(my_computer_fpath + "3a1615c2-2350-46d8-adf9-1415ddad370e.csv")

In [ ]:
# Load the pickled DataFrame
# complete_data = pd.read_pickle("C:\\Users\\dfber\\OneDrive - Mass General Brigham\\Epidural project\\Data\\processed_merlin_data.pkl")
complete_data = pd.read_pickle("C:\\Users\\User\\OneDrive - Mass General Brigham\\Epidural project\\Data\\processed_merlin_data.pkl")

# Now you can work with the DataFrame
complete_data.head()

In [ ]:
df = complete_data.copy()

In [ ]:
ayumi_df = pd.read_excel("C:\\Users\\User\\OneDrive - Mass General Brigham\\Epidural project\\Data\\ayumi_data.xlsx")

In [ ]:
raw_identified_data = pd.read_csv("C:\\Users\\User\OneDrive - Mass General Brigham\\Epidural project\\Data\\Full Identified raw anesthesia_procedure_notes.csv")

# Look for separate vs concatenated notes at other hospitals

We can see here that before the end of 2016, concatenated notes stop appearing. The few that appear afterwards are not true concatenations but times when someone free-text commented on airway during an epidural note or vice versa

In [ ]:
# Step 1: Filter NoteTXT for "epidural" and "airway" (case-insensitive) - this looks for double-notes but also will find single notes that contain both terms
mask = (
    raw_identified_data['NoteTXT'].str.contains('epidural', case=False, na=False) &
    raw_identified_data['NoteTXT'].str.contains('airway', case=False, na=False)
)
filtered_notes = raw_identified_data[mask][['PatientEncounterID', 'epic_pmrn']]

# Step 2: Split and explode anes_procedure_encounter_id column
merlin_df_exploded = merlin_df.copy()
merlin_df_exploded['anes_procedure_encounter_id_2273'] = merlin_df_exploded[
    'anes_procedure_encounter_id_2273'
].astype(str).str.split('|')
merlin_df_exploded = merlin_df_exploded.explode('anes_procedure_encounter_id_2273')

# Optional: strip whitespace if needed
merlin_df_exploded['anes_procedure_encounter_id_2273'] = merlin_df_exploded[
    'anes_procedure_encounter_id_2273'
].str.strip()

# Step 3: Merge with filtered notes
merged = pd.merge(
    filtered_notes,
    merlin_df_exploded.drop('epic_pmrn', axis=1),  # Drop epic_pmrn to avoid duplication
    left_on='PatientEncounterID',
    right_on='anes_procedure_encounter_id_2273',
    how='inner'
)

# # Step 4: Filter for delivery_site == 'nwh'
# merged = merged[merged['delivery_site_2188'].str.lower() == 'nwh']

# Step 5: Sort by delivery_date
merged = merged.sort_values(by='delivery_date')

# Step 6: Print final desired columns
print(merged[['PatientEncounterID', 'delivery_date', 'epic_pmrn']])


In [ ]:
merged[['PatientEncounterID', 'delivery_date', 'epic_pmrn','delivery_site_2188']]

# Compare my data to Ayumi's

In [ ]:
ayumi_df

Directly compare Merlin and Ayumi to see how many catheters are in both

In [ ]:
# Ensure strings
ayumi_df['MRN'] = ayumi_df['MRN'].astype(str)
merlin_df['epidural_bwh_mrns_2354'] = merlin_df['epidural_bwh_mrns_2354'].astype(str)

# Collect matching rows manually
matches = []

for _, row in ayumi_df.iterrows():
    mrn = row['MRN']
    matched_merlin = merlin_df[merlin_df['epidural_bwh_mrns_2354'].str.contains(mrn, na=False)]
    
    # For each match, combine with the ayumi row
    for _, merlin_row in matched_merlin.iterrows():
        combined = pd.concat([row, merlin_row])
        matches.append(combined)

# Convert to DataFrame
merged = pd.DataFrame(matches)


In [ ]:
len(ayumi_df['MRN'].unique()), len(merlin_df['epidural_bwh_mrns_2354'].unique()), len(merged['MRN'].unique()), len(merged['epidural_bwh_mrns_2354'].unique())

Now compare to my processed data set

Note that PMRNs from here do not translate to Epic

In [ ]:
# Ensure strings
ayumi_df['MRN'] = ayumi_df['MRN'].astype(str)
df['epidural_bwh_mrns_2354'] = df['epidural_bwh_mrns_2354'].astype(str)

# Collect matching rows manually
matches = []

for _, row in ayumi_df.iterrows():
    mrn = row['MRN']
    matched_rows = df[df['epidural_bwh_mrns_2354'].str.contains(mrn, na=False)]
    
    # For each match, combine with the ayumi row
    for _, match_row in matched_rows.iterrows():
        combined = pd.concat([row, match_row])
        matches.append(combined)

# Convert to DataFrame
merged = pd.DataFrame(matches)


In [ ]:
len(ayumi_df['MRN'].unique()), len(df['epidural_bwh_mrns_2354'].unique()), len(merged['MRN'].unique()), len(merged['epidural_bwh_mrns_2354'].unique())

Need to remove pre-2017 as these are no longer in my data set

In [ ]:
ayumi_df['Time'] = pd.to_datetime(ayumi_df['Time'], errors='coerce')
ayumi_df_post_2017 = ayumi_df[ayumi_df['Time'] > pd.Timestamp('2017-01-01')]
# Ensure strings
ayumi_df_post_2017['MRN'] = ayumi_df_post_2017['MRN'].astype(str)
df['epidural_bwh_mrns_2354'] = df['epidural_bwh_mrns_2354'].astype(str)

# Collect matching rows manually
matches = []

for _, row in ayumi_df_post_2017.iterrows():
    mrn = row['MRN']
    matched_rows = df[df['epidural_bwh_mrns_2354'].str.contains(mrn, na=False)]
    
    # For each match, combine with the ayumi row
    for _, match_row in matched_rows.iterrows():
        combined = pd.concat([row, match_row])
        matches.append(combined)

# Convert to DataFrame
merged = pd.DataFrame(matches)

In [ ]:
len(ayumi_df_post_2017['MRN'].unique()), len(df['epidural_bwh_mrns_2354'].unique()), len(merged['MRN'].unique()), len(merged['epidural_bwh_mrns_2354'].unique())

In [ ]:
ayumi_df.shape

In [ ]:
ayumi_df_post_2017.shape

Collapse unique MRNs; if any failure then the collapsed MRN has `failed` is 1

In [ ]:
# Ensure correct types
merged['failed_catheter'] = merged['failed_catheter'].astype(float)

# Convert Time and delivery_date to string (and drop NaT safely)
merged['Time'] = merged['Time'].astype(str)
merged['delivery_date'] = merged['delivery_date'].astype(str)

# Collapse to one row per MRN with failed + concatenated dates
collapsed = (
    merged.groupby('MRN')
    .agg({
        'failed_catheter': lambda x: (x == 1).any(),  # True if any failed
        'Time': lambda x: '|'.join(sorted(set(x))),  # concat unique Times
        'delivery_date': lambda x: '|'.join(sorted(set(x))),  # concat unique delivery_dates
        'epic_pmrn': 'first'  # just pick first one (or use a join here too if needed)
    })
    .rename(columns={'failed_catheter': 'failed'})
    .reset_index()
)

In [ ]:
collapsed

In [ ]:
collapsed.shape

In [ ]:
collapsed['failed'].sum()

In [ ]:
collapsed[collapsed['failed'] == False] # Catheters that are marked as failed by Ayumi but marked as not-failed by me:

Catheters that are marked as failed by Ayumi but marked as not-failed by me:
1) The anesthesia encounter was terminated and a new one was made for the repeat epidural 10080675744
2) The repeat catheter was within 10 minutes of the first one 10040815687
3) Epidural note not captured in Merlin 10052565972
4) Not actually a failure! 10091378998
5) Repeat catheter placed within 10 minutes of the first one 10131236735
6) Epidural note not captured in Merlin 10097511113

In [ ]:
# Step 1: Filter delivery date
df['delivery_datetime'] = pd.to_datetime(df['delivery_datetime'], errors='coerce')
df_filtered = df[df['best_timestamp'] < pd.Timestamp('2021-01-01',tz='America/New_York')]
df_filtered = df_filtered[df_filtered['delivery_site'] == 'bwh']

# Step 2: Keep only failed_catheter == 1
df_filtered = df_filtered[df_filtered['failed_catheter'] == 1]

# Step 3: Prepare ayumi MRNs as a set for fast lookup
ayumi_mrns = set(ayumi_df['MRN'].astype(str))

# Step 4: Check for match in any |-delimited MRN
def get_match_info(epi_str):
    if pd.isna(epi_str):
        return False, None
    epidural_mrns = epi_str.split('|')
    for mrn in epidural_mrns:
        if mrn in ayumi_mrns:
            return True, mrn
    return False, None


df_filtered[['has_ayumi_match', 'matching_ayumi_mrn']] = df_filtered['epidural_bwh_mrns_2354'] \
    .astype(str) \
    .apply(lambda x: pd.Series(get_match_info(x)))


In [ ]:
df_filtered.shape

In [ ]:
df_filtered[df_filtered['has_ayumi_match']].shape

In [ ]:
df_filtered[~df_filtered['has_ayumi_match']][['epidural_bwh_mrns_2354', 'delivery_datetime', 'best_timestamp']] # Catheters that are marked as failed by Ayumi but marked as not-failed by me:

These seem generally to be true failures that were missed by Ayumi for whatever reason

# Manually analyze some successes and failures

In [ ]:
raw_identified_data.loc[raw_identified_data['NoteID'] == '2362576456']

In [ ]:
# Create a mapping of 'NoteID' to 'epic_pmrn' from raw_identified_data
note_to_mrn = raw_identified_data.set_index('NoteID')['epic_pmrn'].to_dict()
note_to_purpose = raw_identified_data.set_index('NoteID')['NotePurposeDSC'].to_dict()

# Use the mapping to create the new 'mrn' column in df
df['epic_pmrn'] = df['anes_procedure_note_id_2260'].map(note_to_mrn)
df['NotePurposeDSC'] = df['anes_procedure_note_id_2260'].map(note_to_purpose)

In [ ]:
# prompt: Choose 10 random failed_catheters and 10 random non-failed_catheters

# Assuming 'df' is your DataFrame and it contains a column 'failed_catheter'
failed_catheters = df[df['failed_catheter'] == 1]
non_failed_catheters = df[df['failed_catheter'] == 0]

# Randomly choose 10 failed catheters
random_failed_catheters = failed_catheters.sample(n=10, random_state=42)  # random_state for reproducibility
chosen_failed_catheter_encounter_ids = ['3324914343','3272008150','3234765502','3305371022','3216449190','3186345033','3493903332','3285273066','3320528828','3191160118']
chosen_failed_catheters = df[df['anes_procedure_encounter_id_2273'].isin(chosen_failed_catheter_encounter_ids)]

# Randomly choose 10 non-failed catheters
random_non_failed_catheters = non_failed_catheters.sample(n=10, random_state=42) # random_state for reproducibility

In [ ]:
column_names = [
    'epic_pmrn',
    "best_timestamp",
    "failed_catheter",
    "true_procedure_type",
    "NotePurposeDSC",
    "Regulated_Anesthesiologist_Name",
    "Regulated_Resident_Name",
    "anes_procedure_encounter_id_2273",
    "anes_procedure_note_id_2260",
    "subsequent_proof_of_failure_note_id",
]

In [ ]:
df['true_procedure_type'].value_counts()

In [ ]:
random_failed_catheters[column_names]

In [ ]:
chosen_failed_catheters[column_names]

In [ ]:
random_non_failed_catheters[column_names]

In [ ]:
df[df['anes_procedure_encounter_id_2273'] == '3191160118'][column_names]

In [ ]:
# Filter the DataFrame for failed catheters and delivery location 'mgh'
random_failed_catheters_mgh = df[(df['failed_catheter'] == True) & (df['delivery_site_2188'] == 'mgh')].sample(n=10, random_state=42)

# Display the chosen sample
random_failed_catheters_mgh[column_names]
chosen_failed_catheters_mgh_encounter_ids = ["3268447806", "3396191507", "3258959083", "3581696894", "3271964781", "3583787789", "3402989492", "3476124055", "3304131417", "3522418740"]
chosen_failed_catheter_mgh_note_ids = ['2903598031', '6426160113', '2535157730', '11282242570',
       '3002237621', '11340428769', '6612736939', '8559605944',
       '3947063203', '9788012155']

In [ ]:
df[df['anes_procedure_encounter_id_2273'].isin(chosen_failed_catheters_mgh_encounter_ids)][column_names]

In [ ]:
df[df['anes_procedure_note_id_2260'].isin(chosen_failed_catheter_mgh_note_ids)][column_names]

In [ ]:
random_failed_catheters_mgh[column_names]

# Look at cases where a failure is replaced by the same anesthesia team

Encounter_ID 3607123568 is an example where the same attending/resident team did the index procedure and the replacement (in this case because the epidural migrated out)

In [ ]:
def check_if_replacement_has_same_anesthesia_team(row, df):
    if row['is_neuraxial_catheter'] and row['failed_catheter']:
        this_anesthesiologist = row['Regulated_Anesthesiologist_Name']
        this_resident = row['Regulated_Resident_Name']
        subsequent_procedures = row['subsequent_proof_of_failure_note_id'].replace('\'','').replace('[','').replace(']','').split(',')
        for note_id in subsequent_procedures:
            replacing_anesthesiologist = df[(df['anes_procedure_note_id_2260'] == note_id) & (df['is_neuraxial_catheter'] == True)]['Regulated_Anesthesiologist_Name']
            replacing_resident = df[(df['anes_procedure_note_id_2260'] == note_id) & (df['is_neuraxial_catheter'] == True)]['Regulated_Resident_Name']
            if (this_anesthesiologist == replacing_anesthesiologist).any() and (this_resident == replacing_resident).any():
                # print()
                # print('new note')
                # print(this_anesthesiologist)
                # print(this_resident)
                # print(replacing_anesthesiologist)
                # print(replacing_resident)
                # print(this_anesthesiologist == replacing_anesthesiologist)
                # print(this_resident == replacing_resident)
                row['replaced_by_same_team'] = True


In [ ]:
df[df['failed_catheter']==1].apply(lambda x: check_if_replacement_has_same_anesthesia_team(x, df), axis=1)